In [4]:
from keras.layers.core import Activation, Dense, Dropout
from keras.layers import concatenate,Reshape,Add,LSTM,Multiply
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential,Model
from keras import Model
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras import Input
import keras
import numpy as np
import multiprocessing
import collections
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.preprocessing.text import text_to_word_sequence
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import string
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from time import time
from gensim.models import Word2Vec
from keras.initializers import Constant


## Define parameters

In [108]:
seqLen=100
dateShape=(10,)
signShape=(10,)
textShape=(10,10)

## Define structure

### Date input

In [109]:
inputDate=Input(shape=dateShape)
denseD1=Dense(8,activation="relu")(inputDate)
outputD=Dense(16,activation="relu")(denseD1)

### Sign input

In [110]:
inputSign=Input(shape=signShape)
denseS1=Dense(8,activation="relu")(inputSign)
outputS=Dense(16,activation="relu")(denseS1)

### Text input

In [111]:
inputText=Input(shape=textShape)

### Concatenate date and sign

In [112]:
combined=concatenate([outputF,outputS])

## input->add tanh->lstm->mul sigm->lstm

### Define tanh dense

In [113]:
denseTanh=Dense(100,activation='tanh')(combined)
reshapedTanh=Reshape(textShape)(denseTanh)

### Define sigmoid dense

In [114]:
denseSigmoid=Dense(100,activation='sigmoid')(combined)
reshapedSigmoid=Reshape(textShape)(denseSigmoid)

### Add 

In [115]:
add1=Add()([reshapedTanh,inputText])
lstm1=LSTM(100)(add1)
reshapedLstm=Reshape(textShape)(lstm1)
mul1=Multiply()([reshapedLstm,reshapedSigmoid])
lstm2=LSTM(100)(mul1)

## Define model

In [1]:
#model=Model(inputs=[inputDate,inputSign,inputText],outputs=[lstm2])

In [3]:
w2v = Word2Vec.load("word2vec.model")

In [ ]:
#Подготавливаем embedding слой для использования в модели
#В матрице embedding номер слова заменяется на вектор из модели word2vec  
def getEmbeddingLayer(num_words, embedding_size, max_length, tokenizer, word2vec, Trainable = False):
  print("Размер embedding матрицы:", num_words, "x", embedding_size)
  embedding_matrix = np.zeros((num_words, embedding_size))
  for word, i in tokenizer.word_index.items():
    if i > num_words: #если индекс превышает кол-во слов в словаре, то скипаем  
      continue
    embedding_vector = w2v[word] #получаем вектор соответствущий слову в модели word2vec
    if embedding_vector is not None:  #если слово отсутствует в словаре word2vec, то оно в матрице np.zeroes останется равным 0
      embedding_matrix[i] = embedding_vector #если слово найдено в словаре токенизатора, то в embedding_matrix проставляем вектор соответствующий слову
      embedding_layer = Embedding(input_dim = num_words, 
                              output_dim = embedding_size, 
                              embeddings_initializer = Constant(embedding_matrix),
                              input_length = max_length, 
                              trainable = Trainable)    
  return embedding_layer, embedding_matrix  

In [ ]:
writers_embedding_layer, writers_embedding_matrix = getEmbeddingLayer(
    num_words,
    EMBEDDING_DIM,
    max_len, 
    tokenizer,
    writers_w2v, 
    Trainable = False)